In [0]:
use catalog my_std_catalog;
use schema my_schema_or_db;

drop table if exists historical_data_copy_contact_schema_mismatch;

create table historical_data_copy_contact_schema_mismatch (
  id string,
  name string,
  email string
);

Below python wont work

In [0]:
%python
from pyspark.sql.functions import lit

df = spark.read.format('csv').option('header', 'true').load(
  'dbfs:/Volumes/my_std_catalog/my_schema_or_db/contact/sample*.csv'
)

for col_name in ['id', 'name', 'email']:
    if col_name not in df.columns:
        df = df.withColumn(col_name, lit(None))

df = df.select('id', 'name', 'email')

df.write.format('delta').mode('append').option('mergeSchema', 'true').saveAsTable(
  'my_std_catalog.my_schema_or_db.historical_data_copy_contact_schema_mismatch'
)

display(df)

You ***cannot*** run DDL & DML statements in the same cell. This is a platform limitation to ensure transactional consistency and proper statement parsing.

In [0]:
COPY INTO historical_data_copy_contact_schema_mismatch
FROM '/Volumes/my_std_catalog/my_schema_or_db/contact'
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true', 'columnMapping' = 'name');

In [0]:
select * from historical_data_copy_contact_schema_mismatch;